Variables to be updated/configured:

In [ ]:
WES = False # False if running for the larger epilepsy-autism multiplex network, True if running for the WES multiplex network
if WES:
    FIGURES_DIR = "figures_wes" # path to directory where figures will the saved (creates the directory if it doesn't exist)
else:
    FIGURES_DIR = "figures" # path to directory where figures will the saved (creates the directory if it doesn't exist)
GRAPH_DIR = "gexf_files" # path to directory where the .gexf files are located
INFO_DIR = "network_info" # path to directory with information on each gene/node in the multiplex network

In [ ]:
# network packages
import networkx as nx
from networkx.algorithms.operators.binary import intersection
from networkx.generators.degree_seq import expected_degree_graph
from networkx.readwrite.gexf import read_gexf
from cdlib import evaluation
import igraph as ig
import louvain

# visualization packages
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# other packages
from math import log10, floor
import os
import numpy as np
import pandas as pd

# stats packages    
import scikit_posthocs as sp

from scipy.optimize import curve_fit
from scipy import stats

import uncertainties.unumpy as unp
import uncertainties as unc


In [ ]:
if not os.path.exists(FIGURES_DIR):
    os.makedirs(FIGURES_DIR)

In [ ]:
# plotting settings
font = {'size': 14}
matplotlib.rc('font', **font)

# Read data and get statistics

In [ ]:
if WES:
    gene_phenotype_filename = 'gene-phenotype-wes-1-500-update.gexf'
    gene_ppi_filename = "gene-ppi-wes-700-update.gexf"
    gene_union_filename = 'gene-union-wes.gexf'
    info_filename = 'info_wes_df.csv'
else:
    gene_phenotype_filename = 'gene-phenotype-1-1000-update.gexf'
    gene_ppi_filename = 'gene-ppi-700-update.gexf'
    gene_union_filename = 'gene-union.gexf'
    info_filename = 'info_df.csv'
    
info_df = pd.read_csv(os.path.join(INFO_DIR, info_filename))
gene_phenotype = read_gexf(os.path.join(GRAPH_DIR, gene_phenotype_filename))
gene_ppi = read_gexf(os.path.join(GRAPH_DIR, gene_ppi_filename))
gene_union = read_gexf(os.path.join(GRAPH_DIR, gene_union_filename))

In [ ]:
print('Gene PPI layer')
print(nx.info(gene_ppi))

In [ ]:
print('Gene Phenotype layer')
print(nx.info(gene_phenotype))

# Degree distribution of PPI and phenotype layers

In [ ]:
gene_ppi_degree = info_df['ppi_degree']
gene_phenotype_degree = info_df['phenotype_degree']

In [ ]:
if WES:
    bins = 15
else:
    bins = 30

plt.figure(figsize=(8,6))
plt.hist(gene_ppi_degree, bins=bins, density=True, label='PPI', alpha=0.5)
plt.hist(gene_phenotype_degree, bins=bins, density=True, label='Phenotype', alpha=0.5)

plt.legend()
plt.xlabel('Degree')
plt.ylabel('Frequency')

plt.savefig(os.path.join(FIGURES_DIR, "degree_distribution_ppi_and_phenotype.eps"), dpi=600)

plt.show()

# Significance of overlapping edges between PPI and phenotype layers

In [ ]:
I = intersection(gene_phenotype, gene_ppi)
overlapping_edges = len(I.edges)

In [ ]:
num_trials = 10000
edge_overlap = []
for i in range(0, num_trials):
    if i % 100 == 0:
        print("Trial:", i)
    gene_phenotype_random = expected_degree_graph(gene_phenotype_degree, seed=None, selfloops=False)
    gene_ppi_random = expected_degree_graph(gene_ppi_degree, seed=None, selfloops=False) 
    I_random = intersection(gene_phenotype_random, gene_ppi_random)
    edge_overlap.append(len(I_random.edges))

In [ ]:
# significant number of overlapping edges
plt.figure(figsize=(8, 6))
plt.hist(edge_overlap, bins=15, density=True, label='edge_overlap')
plt.axvline(overlapping_edges, color='red')
plt.ylabel('Frequency')
plt.xlabel('Number of overlapping edges')
plt.savefig(os.path.join(FIGURES_DIR, "overlapping_edges.eps"), dpi=600)
plt.show()

# Phenotype and PPI degree correlation

In [ ]:
# code copied from https://apmonitor.com/che263/index.php/Main/PythonRegressionStatistics

plt.figure(figsize=(8,6))

x = info_df['phenotype_degree']
y = info_df['ppi_degree']
n = len(y)

def f(x, a, b):
    return a * x + b

popt, pcov = curve_fit(f, x, y)

# retrieve parameter values
a = popt[0]
b = popt[1]
print('Optimal Values')
print('a: ' + str(a))
print('b: ' + str(b))

# compute r^2
r2 = 1.0-(sum((y-f(x,a,b))**2)/((n-1.0)*np.var(y,ddof=1)))
print('R^2: ' + str(r2))

# calculate parameter confidence interval
a,b = unc.correlated_values(popt, pcov)
print('Uncertainty')
print('a: ' + str(a))
print('b: ' + str(b))

# plot data
plt.scatter(x, y, s=3, alpha=0.5)

# calculate regression confidence interval
if WES:
    px = np.linspace(0, 20, 100)
else:
    px = np.linspace(0, 150, 100)
    
py = a*px+b
nom = unp.nominal_values(py)
std = unp.std_devs(py)

def predband(x, xd, yd, p, func, conf=0.95):
    # x = requested points
    # xd = x data
    # yd = y data
    # p = parameters
    # func = function name
    alpha = 1.0 - conf    # significance
    N = xd.size          # data sample size
    var_n = len(p)  # number of parameters
    # Quantile of Student's t distribution for p=(1-alpha/2)
    q = stats.t.ppf(1.0 - alpha / 2.0, N - var_n)
    # Stdev of an individual measurement
    se = np.sqrt(1. / (N - var_n) * \
                 np.sum((yd - func(xd, *p)) ** 2))
    # Auxiliary definitions
    sx = (x - xd.mean()) ** 2
    sxd = np.sum((xd - xd.mean()) ** 2)
    # Predicted values (best-fit model)
    yp = func(x, *p)
    # Prediction band
    dy = q * se * np.sqrt(1.0+ (1.0/N) + (sx/sxd))
    # Upper & lower prediction bands.
    lpb, upb = yp - dy, yp + dy
    return lpb, upb

lpb, upb = predband(px, x, y, popt, f, conf=0.95)

# plot the regression
plt.plot(px, nom, c='black', label='fit')

# uncertainty lines (95% confidence)
plt.plot(px, nom - 1.96 * std, c='red',\
         label='95% confidence region')
plt.plot(px, nom + 1.96 * std, c='red')
# prediction band (95% confidence)
plt.plot(px, lpb, 'k--',label='95% prediction band')
plt.plot(px, upb, 'k--')
plt.ylabel('PPI degree')
plt.xlabel('Phenotype degree')
plt.legend(loc='best', fontsize=12)
plt.savefig(os.path.join(FIGURES_DIR, "degree_correlation.eps"), dpi=600)

# save and show figure
plt.show()

# Similarity of modules between PPI and phenotype layer

In [ ]:
# wrapper for communities
class Coms:
    def __init__(self, communities):
        self.communities = communities
        self.overlap = None
        
# get Coms class with genes from annotated networkx graph
def get_coms_from_graph(G):    
    max_module = max([G.nodes[node]['module'] for node in G.nodes])
    partition = []
    for i in range(max_module):
        partition.append([])
    for node in G.nodes:
        mod = G.nodes[node]['module']
        partition[mod-1] = partition[mod-1] + [node]
    coms = Coms(partition)
    return coms

# takes partition with IDs and converts to Coms class with genes
def partition_to_genes(partition):
    partition_genes = []
    for com in partition:
        com_genes = []
        for g in com:
            com_genes.append(id_to_gene[g])
        partition_genes.append(com_genes) 
    coms = Coms(list(partition_genes))
    return coms

In [ ]:
def networkx_to_igraph(G, d):
    g = ig.Graph()
    g.add_vertices([d[g] for g in set(G.nodes)])
    g.add_edges([(d[e[0]], d[e[1]]) for e in set(G.edges)])    
    return g

In [ ]:
gene_to_id = pd.Series(info_df.index.values,index=info_df['gene']).to_dict()
id_to_gene = pd.Series(info_df['gene'].values).to_dict()

In [ ]:
coms_phenotype = get_coms_from_graph(gene_phenotype)
coms_ppi = get_coms_from_graph(gene_ppi)
coms_multiplex = get_coms_from_graph(gene_union)

In [ ]:
mod_phenotype = evaluation.newman_girvan_modularity(gene_phenotype, coms_phenotype)
print("Phenotype layer modularity:", round(mod_phenotype.score, 3))
print("Total number of phenotype modules:", len(coms_phenotype.communities))
print("Total number of phenotype modules with at least 20 genes:", len([com for com in coms_phenotype.communities if len(com)>=20]))
print("Total number of phenotype modules with at least 5 genes:", len([com for com in coms_phenotype.communities if len(com)>=5]))
print()
mod_ppi = evaluation.newman_girvan_modularity(gene_ppi, coms_ppi)
print("PPI layer modularity:", round(mod_ppi.score, 3))
print("Total number of PPI modules:", len(coms_ppi.communities))
print("Total number of PPI modules with at least 20 genes:", len([com for com in coms_ppi.communities if len(com)>=20]))
print("Total number of PPI modules with at least 5 genes:", len([com for com in coms_ppi.communities if len(com)>=5]))


In [ ]:
similarity_calc = evaluation.normalized_mutual_information(coms_phenotype, coms_ppi).score
print('Normalized mutual information of PPI and phenotype layer =', similarity_calc)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
similarity = []
num_trials = 1000
for i in range(0, num_trials):
    if i % 50 == 0:
        print("Trial =", i)
    gene_phenotype_random = expected_degree_graph(gene_phenotype_degree, seed=None, selfloops=False)
    gene_ppi_random = expected_degree_graph(gene_ppi_degree, seed=None, selfloops=False)
    
    # networkx to igraph
    gene_phenotype_random = networkx_to_igraph(gene_phenotype_random, {k:k for k in gene_to_id.values()})
    gene_ppi_random = networkx_to_igraph(gene_ppi_random, {k:k for k in gene_to_id.values()})
    
    partition_phenotype_random = louvain.find_partition(gene_phenotype_random, louvain.ModularityVertexPartition)
    partition_ppi_random = louvain.find_partition(gene_ppi_random, louvain.ModularityVertexPartition)
    
    coms_phenotype_random = partition_to_genes(partition_phenotype_random)
    coms_ppi_random = partition_to_genes(partition_ppi_random)

    #jaccard.append(calculate_jaccard_index_modules(coms_phenotype_random, coms_ppi_random))
    similarity.append(evaluation.normalized_mutual_information(coms_phenotype_random, coms_ppi_random).score)

In [ ]:
# significant number of overlapping modules
plt.figure(figsize=(8, 6))
plt.hist(similarity, bins=20, density=True, label='normalized mutual information')
plt.axvline(similarity_calc, color='red')

plt.ylabel('Frequency')
plt.xlabel('Normalized mutual information')
plt.savefig(os.path.join(FIGURES_DIR, "NMI.eps"), dpi=600)

plt.show()

# Centrality and of epilepsy- and autism-specific genes and common genes

In [ ]:
def round_sig(x, sig=3):
    return round(x, sig-int(floor(log10(abs(x))))-1)

In [ ]:
temp = []
for i, row in info_df.iterrows():        
    if row['common_all'] == 1:
        temp.append(list(row.values) + ['common_all'])
    if row['a_specific'] == 1:
        temp.append(list(row.values) + ['autism_specific'])
    if row['e_specific'] == 1:
        temp.append(list(row.values) + ['epilepsy_specific'])
df = pd.DataFrame(temp)
df.columns = list(info_df.columns) + ['type']

In [ ]:
degree_info = df.melt(id_vars=['gene', 'type'], value_vars=['ppi_degree', 'phenotype_degree'], var_name="graph", value_name="degree")
degree_info = degree_info.replace({'graph': {'ppi_degree': 'PPI', 'phenotype_degree': 'Phenotype'}})
bc_info = df.melt(id_vars=['gene', 'type'], value_vars=['ppi_betweenness', 'phenotype_betweenness'], var_name="graph", value_name="betweenness")
bc_info = bc_info.replace({'graph': {'ppi_betweenness': 'PPI', 'phenotype_betweenness': 'Phenotype'}})

In [ ]:
# PPI degree Kruskal–Wallis H test
var = 'ppi_degree'
x = list(df[df['type']=='epilepsy_specific'][var])
y = list(df[df['type']=='autism_specific'][var])
z = list(df[df['type']=='common_all'][var])

stat, pval = stats.kruskal(x, y, z)
print(f"Statistic = {round_sig(stat)}, p-value = {round_sig(pval)}")
ppi_degree = sp.posthoc_mannwhitney([x, y, z], p_adjust='bonferroni')

In [ ]:
# Phenotype degree Kruskal–Wallis H test
var = 'phenotype_degree'
x = list(df[df['type']=='epilepsy_specific'][var])
y = list(df[df['type']=='autism_specific'][var])
z = list(df[df['type']=='common_all'][var])

stat, pval = stats.kruskal(x, y, z)
print(f"Statistic = {round_sig(stat)}, p-value = {round_sig(pval)}")
phenotype_degree = sp.posthoc_mannwhitney([x, y, z], p_adjust='bonferroni')

In [ ]:
# PPI betweenness centrality (BC) Kruskal–Wallis H test
var = 'ppi_betweenness'
x = list(df[df['type']=='epilepsy_specific'][var])
y = list(df[df['type']=='autism_specific'][var])
z = list(df[df['type']=='common_all'][var])

stat, pval = stats.kruskal(x, y, z)
print(f"Statistic = {round_sig(stat)}, p-value = {round_sig(pval)}")
ppi_bc = sp.posthoc_mannwhitney([x, y, z], p_adjust='bonferroni')

In [ ]:
# Phenotype betweenness centrality (BC) Kruskal–Wallis H test
var = 'phenotype_betweenness'
x = list(df[df['type']=='epilepsy_specific'][var])
y = list(df[df['type']=='autism_specific'][var])
z = list(df[df['type']=='common_all'][var])

stat, pval = stats.kruskal(x, y, z)
print(f"Statistic = {round_sig(stat)}, p-value = {round_sig(pval)}")
phenotype_bc = sp.posthoc_mannwhitney([x, y, z], p_adjust='bonferroni')

In [ ]:
degree_info_ppi = degree_info[degree_info['graph']=='PPI'].groupby(['type'])['degree'].agg(['mean', stats.sem])
degree_info_ppi['order'] = [1, 2, 0]
degree_info_ppi = degree_info_ppi.sort_values(by='order')
degree_info_phenotype = degree_info[degree_info['graph']=='Phenotype'].groupby(['type'])['degree'].agg(['mean', stats.sem])
degree_info_phenotype['order'] = [1, 2, 0]
degree_info_phenotype = degree_info_phenotype.sort_values(by='order')

In [ ]:
# function copied from https://stackoverflow.com/questions/11517986/indicating-the-statistically-significant-difference-in-bar-graph
def barplot_annotate_brackets(num1, num2, data, center, height, yerr=None, dh=.05, barh=.05, fs=None, maxasterix=None):
    """ 
    Annotate barplot with p-values.

    :param num1: number of left bar to put bracket over
    :param num2: number of right bar to put bracket over
    :param data: string to write or number for generating asterixes
    :param center: centers of all bars (like plt.bar() input)
    :param height: heights of all bars (like plt.bar() input)
    :param yerr: yerrs of all bars (like plt.bar() input)
    :param dh: height offset over bar / bar + yerr in axes coordinates (0 to 1)
    :param barh: bar height in axes coordinates (0 to 1)
    :param fs: font size
    :param maxasterix: maximum number of asterixes to write (for very small p-values)
    """

    if type(data) is str:
        text = data
    else:
        # * is p < 0.05
        # ** is p < 0.005
        # *** is p < 0.0005
        # etc.
        text = ''
        p = .05

        while data < p:
            text += '*'
            p /= 10.

            if maxasterix and len(text) == maxasterix:
                break

        if len(text) == 0:
            text = 'n. s.'

    lx, ly = center[num1], height[num1]
    rx, ry = center[num2], height[num2]

    if yerr:
        ly += yerr[num1]
        ry += yerr[num2]

    ax_y0, ax_y1 = plt.gca().get_ylim()
    dh *= (ax_y1 - ax_y0)
    barh *= (ax_y1 - ax_y0)

    y = max(ly, ry) + dh

    barx = [lx, lx, rx, rx]
    bary = [y, y+barh, y+barh, y]
    mid = ((lx+rx)/2, y+barh)

    plt.plot(barx, bary, c='black')

    kwargs = dict(ha='center', va='bottom')
    if fs is not None:
        kwargs['fontsize'] = fs

    plt.text(*mid, text, **kwargs)

In [ ]:
df_ppi = degree_info_ppi

labels = ['Epilepsy-specific', "Autism-specific", "Common"]

degree_ppi_means = list(df_ppi['mean'])
degree_ppi_errors = list(df_ppi['sem'])

x = np.arange(len(degree_info_ppi))  # the label locations
width = 0.35  # the width of the bars

err_kwargs = {'fmt':'none','linewidth':2,'ecolor':'k','capsize':10}

fig, ax = plt.subplots(figsize=(12, 8))
rects1 = ax.bar(x, degree_ppi_means, width, label='PPI')
ax.errorbar(x, degree_ppi_means, yerr=degree_ppi_errors, **err_kwargs)

alpha = 0.05
d = {0: 0.3, 1: 0.1, 2: 0.1}
for i in range(0, len(x)):
    for j in range(i+1, len(x)):
        p = abs(ppi_degree.iloc[i,j])
        if p < alpha:
            barplot_annotate_brackets(i, j, "p = " + str(round_sig(p, sig=3)), x, degree_ppi_means, dh=d[i])

        
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('PPI degree')
ax.set_xlabel('Gene group')
ax.set_xticks(x)
ax.set_xticklabels(labels)

plt.savefig(os.path.join(FIGURES_DIR, "degree_significance_ppi.eps"), dpi=600)

In [ ]:
df_phenotype = degree_info_phenotype

labels = ['Epilepsy-specific', "Autism-specific", "Common"]

degree_phenotype_means = list(df_phenotype['mean'])
degree_phenotype_errors = list(df_phenotype['sem'])

x = np.arange(len(degree_info_ppi))  # the label locations
width = 0.35  # the width of the bars

err_kwargs = {'fmt':'none','linewidth':2,'ecolor':'k','capsize':10}

fig, ax = plt.subplots(figsize=(12, 8))
rects2 = ax.bar(x, degree_phenotype_means, width, label='Phenotype')
ax.errorbar(x, degree_phenotype_means, yerr=degree_phenotype_errors, **err_kwargs)

alpha = 0.05
d = {0: 0.3, 1: 0.1, 2: 0.1}
for i in range(0, len(x)):
    for j in range(i+1, len(x)):
        p = abs(phenotype_degree.iloc[i,j])
        if p < alpha:
            barplot_annotate_brackets(i, j, "p = " + str(round_sig(p, sig=3)), x, degree_phenotype_means, dh=d[i])
        
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Phenotype degree')
ax.set_xlabel('Gene group')
ax.set_xticks(x)
ax.set_xticklabels(labels)

plt.savefig(os.path.join(FIGURES_DIR, "degree_significance_phenotype.eps"), dpi=600)

In [ ]:
bc_info_ppi = bc_info[bc_info['graph']=='PPI'].groupby(['type'])['betweenness'].agg(['mean', stats.sem])
bc_info_ppi['order'] = [1, 2, 0]
bc_info_ppi = bc_info_ppi.sort_values(by='order')
bc_info_phenotype = bc_info[bc_info['graph']=='Phenotype'].groupby(['type'])['betweenness'].agg(['mean', stats.sem])
bc_info_phenotype['order'] = [1, 2, 0]
bc_info_phenotype = bc_info_phenotype.sort_values(by='order')

In [ ]:
df_ppi = bc_info_ppi

labels = ['Epilepsy-specific', "Autism-specific", "Common"]

degree_ppi_means = list(df_ppi['mean'])
degree_ppi_errors = list(df_ppi['sem'])

x = np.arange(len(degree_info_ppi))  # the label locations
width = 0.35  # the width of the bars

err_kwargs = {'fmt':'none','linewidth':2,'ecolor':'k','capsize':10}

fig, ax = plt.subplots(figsize=(12, 8))
rects1 = ax.bar(x, degree_ppi_means, width, label='PPI')
ax.errorbar(x, degree_ppi_means, yerr=degree_ppi_errors, **err_kwargs)

alpha = 0.05
d = {0: 0.3, 1: 0.1, 2: 0.1}
for i in range(0, len(x)):
    for j in range(i+1, len(x)):
        p = abs(ppi_bc.iloc[i,j])
        if p < alpha:
            barplot_annotate_brackets(i, j, "p = " + str(round_sig(p, sig=3)), x, degree_ppi_means, dh=d[i])

        
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('PPI betweenness centrality')
ax.set_xlabel('Gene group')
ax.set_xticks(x)
ax.set_xticklabels(labels)

plt.savefig(os.path.join(FIGURES_DIR, "BC_significance_ppi.eps"), dpi=600)

In [ ]:
df_phenotype = bc_info_phenotype

labels = ['Epilepsy-specific', "Autism-specific", "Common"]

degree_phenotype_means = list(df_phenotype['mean'])
degree_phenotype_errors = list(df_phenotype['sem'])

x = np.arange(len(degree_info_ppi))  # the label locations
width = 0.35  # the width of the bars

err_kwargs = {'fmt':'none','linewidth':2,'ecolor':'k','capsize':10}

fig, ax = plt.subplots(figsize=(12, 8))
rects2 = ax.bar(x, degree_phenotype_means, width, label='Phenotype')
ax.errorbar(x, degree_phenotype_means, yerr=degree_phenotype_errors, **err_kwargs)

alpha = 0.05
d = {0: 0.3, 1: 0.1, 2: 0.1}
for i in range(0, len(x)):
    for j in range(i+1, len(x)):
        p = abs(phenotype_bc.iloc[i,j])
        if p < alpha:
            barplot_annotate_brackets(i, j, "p = " + str(round_sig(p, sig=3)), x, degree_phenotype_means, dh=d[i])
        
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Phenotype betweenness centrality')
ax.set_xlabel('Gene group')
ax.set_xticks(x)
ax.set_xticklabels(labels)

plt.savefig(os.path.join(FIGURES_DIR, "BC_significance_phenotype.eps"), dpi=600)